In [ ]:
# Import and problem generation
from railway import *

# Define problem parameters
P = 2000
K = 3
timelimit = 900 # 5 minutes
heuristics_timelimit = 300 # 5 minutes

# N = 10; J = 10; T = 10 ; ID =  1
# N = 10; J = 10; T = 50 ; ID =  2
# N = 10; J = 10; T = 100; ID =  3
# N = 10; J = 40; T = 10 ; ID =  4
N = 10; J = 40; T = 50 ; ID =  5
# N = 10; J = 40; T = 100; ID =  6
# N = 10; J = 80; T = 10 ; ID =  7
# N = 10; J = 80; T = 50 ; ID =  8
# N = 10; J = 80; T = 100; ID =  9
# N = 20; J = 10; T = 10 ; ID = 10
# N = 20; J = 10; T = 50 ; ID = 11
# N = 20; J = 10; T = 100; ID = 12
# N = 20; J = 40; T = 10 ; ID = 13
# N = 20; J = 40; T = 50 ; ID = 13
# N = 20; J = 40; T = 100; ID = 15
# N = 20; J = 80; T = 10 ; ID = 16
# N = 20; J = 80; T = 50 ; ID = 17
# N = 20; J = 80; T = 100; ID = 18
# N = 40; J = 10; T = 10 ; ID = 19
# N = 40; J = 10; T = 50 ; ID = 20
# N = 40; J = 10; T = 100; ID = 21
# N = 40; J = 40; T = 10 ; ID = 22
# N = 40; J = 40; T = 50 ; ID = 23
# N = 40; J = 40; T = 100; ID = 24
# N = 40; J = 80; T = 10 ; ID = 25
# N = 40; J = 80; T = 50 ; ID = 26
# N = 40; J = 80; T = 100; ID = 27



# # Create blueprint model
# model = Railway(N, T, J, P, K)

# # Generate problem parameters
# job_min_time, job_max_time = 1, T//4
# job_min_length, job_max_length = 1, 1
# pause_min_time, pause_max_time = 0, 2
# min_demand, max_demand = 0.0, 1.0
# min_share, max_share = 0.1, 0.9
# min_capacity, max_capacity = 0.5, 0.75
# n_max_events = 0
# event_min_length, event_max_length = 0, 0

# model.generate(
#     job_min_time,
#     job_max_time,
#     job_min_length,
#     job_max_length,
#     pause_min_time,
#     pause_max_time,
#     min_demand,
#     max_demand,
#     min_share,
#     max_share,
#     min_capacity,
#     max_capacity,
#     n_max_events,
#     event_min_length,
#     event_max_length,
# )


# Load model

# Name of the file to load
FILENAME = f"../datasets_new/railway_N{N}_T{T}_J{J}_P{P}_K{K}.json"

# Instantiate the Railway object
model = Railway.load(FILENAME)


# Set up the three models by (deep) copying the blueprint model
model0 = Railway.copy(model)
model1 = Railway.copy(model)
model2 = Railway.copy(model)

In [ ]:
# 🛃 Validator
validate = Railway.copy(model)
# validate.model.setParam('OutputFlag', 0)
validate.model.setParam('Presolve', 0)
validate.model.setParam('TimeLimit', 30)
validate.set_constraints()
validate.set_objective()
validresults = validate.optimize()

validresults

In [ ]:
# Save dataset
import os
DATASET_DIR = "../datasets_new"
FILENAME = os.path.join(DATASET_DIR, f"railway_N{N}_T{T}_J{J}_P{P}_K{K}.json") 
model.save(FILENAME)	
print(f"✅ Dataset N={N}, T={T}, J={J}, P={P}, K={K} created")

***

In [ ]:
# del model0
# # model0 = Railway.copy(model)

# # Name of the file to load
# FILENAME = f"../datasets_new/railway_N{N}_T{T}_J{J}_P{P}_K{K}.json"
# model0 = Railway.load(FILENAME)

# print(model0)

In [ ]:
# 0️⃣ Model 0: "as-is" Gurobi model with no heuristics or cuts
print("\nModel 0\n")
model0.set_model0(timelimit, True)
model0.set_constraints()
model0.set_objective()
results0 = model0.optimize()

print(results0)

In [ ]:
# del model1
# model1 = Railway.copy(model)

# print(model1)

In [ ]:
# 1️⃣ Model 1: model with simulated annealing heuristic
print("\nModel 1\n")
model1.set_model1(timelimit, True)
S, SAtime = model1.simulated_annealing(
    T=5e4,
    c=0.99,
    L=1,
    min_T=1,
    max_time=heuristics_timelimit,
    debug = True
)
model1.set_solution(S)

# _, _, _, _, v2 = model1.get_vars_from_times(S)
# heuristics_obj = model1.get_objective_value(v2)
# model1.model.setParam('Cutoff', heuristics_obj)

model1.set_constraints()
model1.set_objective()

results1 = model1.optimize()

print(results1)

In [ ]:
del model2
model2 = Railway.copy(model)

print(model2)

In [ ]:
# 2️⃣ Model 2: Full model plus valid inequalities and cutting planes
print("\nModel 2\n")
# model2.set_model2(timelimit, True)
# print('Running simulated annealing...')
# S, SAtime = model2.simulated_annealing(
#     T=5e4,
#     c=0.99,
#     L=1,
#     min_T=1,
#     max_time=heuristics_timelimit,
#     debug=True
# )

model2.model.setParam('TimeLimit',timelimit)
model2.model.setParam('PoolSolutions', 1)
# model2.model.setParam('Cuts', 0)
# model2.model.setParam('CutPasses', 0)
model2.model.setParam('Heuristics', 0)
model2.model.setParam('Symmetry', 0)
model2.model.setParam('Threads', 1)
model2.model.setParam('Presolve', 0)
model2.model.setParam('NumericFocus', 3)

# model2.model.params.BQPCuts = -1
# model2.model.params.CliqueCuts = -1
# model2.model.params.CoverCuts = -1
# model2.model.params.FlowCoverCuts = -1
# model2.model.params.FlowPathCuts = -1
# model2.model.params.GomoryPasses = -1
# model2.model.params.GUBCoverCuts = -1
# model2.model.params.ImpliedCuts = -1
# model2.model.params.LiftProjectCuts = -1
# model2.model.params.MIRCuts = -1
# model2.model.params.ModKCuts = -1
# model2.model.params.NetworkCuts = -1
# model2.model.params.RelaxLiftCuts = -1
# model2.model.params.StrongCGCuts = -1
# model2.model.params.ZeroHalfCuts = -1

# _, _, _, _, v2 = model2.get_vars_from_times(S)
# heuristics_obj = model2.get_objective_value(v2)
# model2.model.setParam('Cutoff', heuristics_obj)

model2.set_solution(S)

model2.set_constraints()
model2.set_objective()

model2.set_valid_inequalities()
model2.set_cutting_planes()

results2 = model2.optimize()

print(results2)

In [ ]:
# 📊 Summary

print("\nValidator")
print("Runtime:", validresults['runtime'])
print("Gap:", validresults['gap'])
print("Objective value:", validresults['obj'])
print("Nodes:", validresults['nodes'])
print("Iterations:", validresults['iterations'])

print("\nModel0")
print("Runtime:", results0['runtime'])
print("Gap:", results0['gap'])
print("Objective value:", results0['obj'])
print("Nodes:", results0['nodes'])
print("Iterations:", results0['iterations'])

print("\nModel1")
print("Runtime:", results1['runtime'])
print("Gap:", results1['gap'])
print("Objective value:", results1['obj'])
print("Nodes:", results1['nodes'])
print("Iterations:", results1['iterations'])

print("\nModel2")
print("Runtime:", results2['runtime'])
print("Gap:", results2['gap'])
print("Objective value:", results2['obj'])
print("Nodes:", results2['nodes'])
print("Iterations:", results2['iterations'])

***

In [ ]:
# Save dataset
import os
DATASET_DIR = "../datasets_new"
FILENAME = os.path.join(DATASET_DIR, f"railway_N{N}_T{T}_J{J}_P{P}_K{K}.json") 
model.save(FILENAME)	
print(f"✅ Dataset N={N}, T={T}, J={J}, P={P}, K={K} created")

In [ ]:
# Save results
import os
import pandas as pd
RESULTFILE = f"../results/results_new_gen.csv"
columns=[
    'ID',
    'model',
    'N',
    'J',
    'T',
    'P',
    'K',
    'status',
    'runtime',
    'heuristics_time',
    'total_time',
    'gap',
    'objective',
    'nodes',
    'iterations',
    'timelimit',
    'heuristics_limit'
]
try:
    df = pd.read_csv(RESULTFILE)
except FileNotFoundError:
    df = pd.DataFrame( columns=columns)
    df.to_csv(RESULTFILE, index=False)
    print(f"Created {RESULTFILE} file for scalability results.")

row0 = {
    'ID': ID,
    'model': 0,
    'N': N,
    'J': J,
    'T': T,
    'P': P,
    'K': K,
    'status': model0.get_status(),
    'runtime': results0['runtime'],
    'heuristics_time': 0,
    'total_time': results0['runtime'],
    'gap': results0['gap'],
    'objective': results0['obj'],
    'nodes': results0['nodes'],
    'iterations': results0['iterations'],
    'timelimit': timelimit,
    'heuristics_limit': heuristics_timelimit
}

row1 = {
    'ID': ID,
    'model': 1,
    'N': N,
    'J': J,
    'T': T,
    'P': P,
    'K': K,
    'status': model1.get_status(),
    'runtime': results1['runtime'],
    'heuristics_time': SAtime,
    'total_time': SAtime + results1['runtime'],
    'gap': results1['gap'],
    'objective': results1['obj'],
    'nodes': results1['nodes'],
    'iterations': results1['iterations'],
    'timelimit': timelimit,
    'heuristics_limit': heuristics_timelimit
}

row2 = {
    'ID': ID,
    'model': 2,
    'N': N,
    'J': J,
    'T': T,
    'P': P,
    'K': K,
    'status': model2.get_status(),
    'runtime': results2['runtime'],
    'heuristics_time': SAtime,
    'total_time': SAtime + results2['runtime'],
    'gap': results2['gap'],
    'objective': results2['obj'],
    'nodes': results2['nodes'],
    'iterations': results2['iterations'],
    'timelimit': timelimit,
    'heuristics_limit': heuristics_timelimit
}

# Append results to the dataframe
df.loc[len(df)] = row0
df.loc[len(df)] = row1
df.loc[len(df)] = row2

# Save the dataframe to the csv file
df.to_csv(RESULTFILE, index=False)